In [2]:
from nba_api.stats.endpoints import playerindex, playbyplayv3 


In [4]:
def calculate_minutes_played(play_by_play_df, until_clock):
    """
    Calculates the number of minutes played by each player until the given clock timestamp.
    """
    # Parse until_clock into a timedelta for comparison
    #until_time = Timedelta(until_clock, unit='s')

    # Sort the data by period and actionNumber to simulate the game progression
    play_by_play_df = play_by_play_df.sort_values(by=['period', 'actionNumber']).reset_index(drop=True)

    # Track active players on the court for each team
    active_players = {}
    total_play_time = {}

    # Filter out substitution events
    sub_events = play_by_play_df[(play_by_play_df.actionType == 'Substitution') & (play_by_play_df.ABS_TIME <= until_clock)].reset_index(drop=True)

    for _, row in sub_events.iterrows():
        current_time = sub_events['ABS_TIME']
        print(current_time)

        description = row['description']
        if 'SUB' in description:
            # Parse substitution description
            _, sub_in, _, sub_out = description.split()

            # Handle substitution logic for active players
            team = row['teamTricode']

            if team not in active_players:
                active_players[team] = {}

            # Record play time for the player being substituted out
            if sub_out in active_players[team]:
                play_start_time = active_players[team].pop(sub_out)
                minutes_played = (current_time - play_start_time).total_seconds() / 60
                total_play_time[sub_out] = total_play_time.get(sub_out, 0) + minutes_played

            # Add the substituted-in player to active players
            active_players[team][sub_in] = current_time

    # Add remaining play times for players still on the court
    for team, players in active_players.items():
        for player, play_start_time in players.items():
            minutes_played = (until_time - play_start_time).total_seconds() / 60
            total_play_time[player] = total_play_time.get(player, 0) + minutes_played

    # Convert to a DataFrame for easier analysis
    result = pd.DataFrame(list(total_play_time.items()), columns=['Player', 'MinutesPlayed'])
    result = result.sort_values(by='MinutesPlayed', ascending=False).reset_index(drop=True)

    return result


In [6]:
ttp = calculate_minutes_played(test, 720)

NameError: name 'test' is not defined